# Introduction
In previous work, I've noticed that there was no ATP synthase in the model, though Beata did initially include it in her thesis. It is to be expected that *Parageobacillus thermoglucosidasius* NCIB11955 has an ATP synthase as it grows aerobically quite fast and also it is encoded in the genome. Therefore, in this notebook I will add the ATP synthase back into the model. 

In doing so, I will check all transport reactions to ensure there is no free exchange of protons. For transport reactions, I will only include proton transport when we expect there to be symport or antiport or protons. As the transporters are not well studied in our strain, I will take guidance form *Bacillus subtilis*. This is not ideal but we need to do with the information we have. 


In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite

In [2]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [3]:
model_e_coli = cameo.load_model('iML1515')

In [4]:
model_b_sub = cameo.load_model('iYO844')

### Adding ATP synthase

In [5]:
model.add_reaction(Reaction(id='ATPS4r'))

In [6]:
model.reactions.ATPS4r.name = 'ATP synthase (four protons for one ATP)'

In [7]:
model.reactions.ATPS4r.annotation = model_b_sub.reactions.ATPS4r.annotation

In [8]:
model.reactions.ATPS4r.add_metabolites({model.metabolites.adp_c: -1, model.metabolites.h_e:-4.0, model.metabolites.pi_c:-1, model.metabolites.atp_c:1, model.metabolites.h2o_c:1, model.metabolites.h_c:3.0 })

In [9]:
model.reactions.ATPS4r.bounds = (-1000,1000)

In [10]:
#check effect on biomass prediction
model.optimize().objective_value

1.840490745188012

In [11]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

## Checking transport reactions for protons

First of all, we should remove the Ht reaction, as it will just generate an energy forming cycle. 

In [12]:
model.remove_reactions(model.reactions.Ht)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [13]:
transports = []
#collect all transport reactions: either end in t or abc or pts
for rct in model.reactions:
    if rct.id[-1:] in 't':
        transports.append(rct)
    elif rct.id[-3:] in 'abc':
        transports.append(rct)
    elif rct.id[-3:] in 'pts':
        transports.append(rct)
    else:
        continue

In [14]:
#make df and export to csv to go through the whole list
transports_id = []
transports_reactions = []
for rct in transports:
    transports_id.append(rct.id)
    transports_reactions.append(rct.reaction)

In [15]:
df = pd.DataFrame({'ID':transports_id, 'Reaction':transports_reactions})

In [16]:
df.to_csv('../databases/Transport_rcts.csv')

Now I will go through the whole list and inspect how they are incorporated into the B, subtilis and E. coli model, as wel as literature to gain ebtter insight into the transport and if this is associated with protons or not. 

In [17]:
model.reactions.NH4t.bounds = (-1000,1000)

In bacillus, it is known that there are multiple ABC transporters needed for iron transport. I would expect it to be similar for geobacillus. In the end, if we associate it with ATP or not: this will be corrected for in the fitting of the growth associted maintanence, and so shouldn't be too much of an issue.*

In [18]:
model.reactions.FE2t.id = 'FEtabc'

In [19]:
model.reactions.FEtabc.add_metabolites({model.metabolites.atp_c:-1, model.metabolites.h2o_c:-1, model.metabolites.adp_c:1, model.metabolites.h_c:1, model.metabolites.pi_c:1})

In [20]:
model.reactions.H2Ot.bounds =(-1000,1000)

In [21]:
model.reactions.O2t.bounds = (-1000,1000)

In [22]:
model.reactions.CO2t.bounds = (-1000,1000)

an ATP binding transporter has been identified in B. subtilis and also in some geobacillus strains. Therefore it would be plausible that our strain has that too. Again here this will be corrected for in the GAM in the biomass as cobalt is only used for that. 

In [23]:
model.reactions.COBALT2t.add_metabolites({model.metabolites.atp_c:-1, model.metabolites.h2o_c:-1, model.metabolites.adp_c:1, model.metabolites.h_c:1, model.metabolites.pi_c:1})

In [24]:
model.reactions.ETOHt.bounds = (-1000,1000)

In [25]:
model.reactions.ACt.bounds = (-1000,1000)

It seems lactate is transported via reversible proton symport, so this will be changed for both L and D lactate. This is also the case for succinate.

In [26]:
model.reactions.LAC__Dt.add_metabolites({model.metabolites.h_c:-1, model.metabolites.h_e:1})

In [27]:
model.reactions.LAC__Dt.bounds = (-1000,1000)

In [28]:
model.reactions.LAC__Lt.bounds = (-1000,1000)

In [29]:
model.reactions.LAC__Lt.add_metabolites({model.metabolites.h_c:-1, model.metabolites.h_e:1})

In [30]:
model.reactions.SUCCt.bounds = (-1000,1000)

In [31]:
model.reactions.SUCCt.add_metabolites({model.metabolites.h_c:-1, model.metabolites.h_e:1})

For chloride: there is a chloride transported identified in the genomes of our strain as well as bacillus and other Geobacilli. Right now chloride is in the biomass reaction, however considering the tables in Beata's thesis, it shouldn't be. On the other hand, potassium should be in the biomass reaction but isnt. Later I will revist all biomass components anyway, so right now I will just add the potassium transport so this is at least in place when we revisit the biomass reaction. 
According to the genome there is both an antiport and a channel present. I will constrain the antiport, and leave the channel reversible. In theory this could cycle protons, but doing so would dissipate energy unnecessarily and this will be preventing from a growth optimization perspective so shouldn't happen. 

I will just leave the chloride as it will not do anything once we remove it from the biomass reaction anyway. 

In [32]:
model.add_metabolites(Metabolite(id='k_c'))
model.add_metabolites(Metabolite(id='k_e'))

In [33]:
model.metabolites.k_c.name = 'Potassium'
model.metabolites.k_c.compartment = 'c'
model.metabolites.k_c.formula = 'K'
model.metabolites.k_c.annotation = model_e_coli.metabolites.k_c.annotation
model.metabolites.k_c.charge = 1

In [34]:
model.metabolites.k_e.name = 'Potassium'
model.metabolites.k_e.compartment = 'e'
model.metabolites.k_e.formula = 'K'
model.metabolites.k_e.annotation = model_e_coli.metabolites.k_c.annotation
model.metabolites.k_e.charge = 1

In [35]:
model.add_reaction(Reaction(id='Kt'))

In [36]:
model.reactions.Kt.name = 'Potassium transport via antiport'

In [37]:
model.reactions.Kt.add_metabolites({model.metabolites.h_e: -1, model.metabolites.k_c:-1, model.metabolites.h_c:1, model.metabolites.k_e: 1})

In [38]:
model.reactions.Kt.bounds = (0,1000)

In [39]:
model.add_reaction(Reaction(id='Kt2'))

In [40]:
model.reactions.Kt2.name = 'Potassium transport via uniport (facilitated diffusion)'
model.reactions.Kt2.bounds = (-1000,1000)

In [41]:
model.reactions.Kt2.add_metabolites({model.metabolites.k_e:-1, model.metabolites.k_c:1})

In [42]:
#should be reversible proton symport
model.reactions.ASN__Lt.bounds = (-1000,1000)

In [43]:
model.reactions.ASN__Lt.name = 'L-asparagine reversible transport via proton symport'
model.reactions.ASN__Lt.add_metabolites({model.metabolites.h_e:-1, model.metabolites.h_c:1})

For quinone, gthrd and nadp: these metabolites shouldnt even be transported. So for now I will leave their transports, as they will be revisited anyway when we have obtained mass balance.

In [44]:
#make reversible incase we would supplement with TPP
model.reactions.THMTPt.bounds = (-1000,1000)

In [45]:
model.reactions.GLYCt.bounds = (-1000,1000)

In [46]:
#Rhamnose via proton symport
model.reactions.RMNt.bounds = (0,1000)
model.reactions.RMNt.add_metabolites({model.metabolites.h_e:-1, model.metabolites.h_c:1})

In [47]:
#mellibiose via proton symport
model.reactions.MELIBt.bounds = (0,1000)
model.reactions.MELIBt.add_metabolites({model.metabolites.h_e:-1, model.metabolites.h_c:1})

After spending some more time taking a look at it, it is more reasonable to expect the pyruvate transport to be coupled to proton symport. 

In [48]:
model.reactions.PYRt.name = 'Pyruvate transport in via proton symport'
model.reactions.PYRt.add_metabolites({model.metabolites.h_e:-1, model.metabolites.h_c:1})

In [49]:
#formate via proton symport
model.reactions.FORt.name = 'Formate transport in via proton symport'
model.reactions.FORt.bounds = (-1000,1000)

In [50]:
#phosphate tranport via symport
model.reactions.PIt.add_metabolites({model.metabolites.h_e:-1, model.metabolites.h_c:1})

In [51]:
model.optimize()

,fluxes,reduced_costs
IDPh,23.900290,-1.232595e-32
CAT,0.000000,-7.104757e-17
PDHam1hi,0.000000,-6.165544e-02
CCP,0.000000,-7.104757e-17
HYDA,0.000000,0.000000e+00
...,...,...
ALATA_L,0.000000,0.000000e+00
HCO3E,6.338725,0.000000e+00
ATPS4r,42.702306,-5.551115e-17
Kt,0.000000,-9.251859e-18


In [52]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')